# Day 20
https://adventofcode.com/2018/day/20

In [1]:
import aocd
data = aocd.get_data(year=2018, day=20)

In [2]:
from collections import deque
from dataclasses import dataclass

In [3]:
@dataclass(frozen=True)
class Point():
    y: int
    x: int
    
    def __add__(self, other):
        return Point(self.y + other.y, self.x + other.x)

In [4]:
directions = {
    'N': Point(-1, 0),
    'E': Point(0, 1),
    'S': Point(1, 0),
    'W': Point(0, -1),
}

In [5]:
def find_all_doors(route_regex):
    doors = set()
    pos = {Point(0, 0)}
    stack = []
    starts, ends = {Point(0, 0)}, set()
    
    for char in route_regex:
        if char == '|':
            # alternate route: update possible ending points and restart the group
            ends.update(pos)
            pos = starts
        elif char == '(':
            # start of group: add current positions as start of a new group
            stack.append((starts, ends))
            starts, ends = pos, set()
        elif char == ')':
            # end of group: finish current group, add current positions as possible ends
            pos.update(ends)
            starts, ends = stack.pop()
        elif char in directions:
            # move in a given direction from all possible current locations
            direction = directions[char]
            doors.update((p, p+direction) for p in pos)
            pos = {p+direction for p in pos}
    
    return doors

In [6]:
def reachability_dict(doors):
    reachability = dict()
    
    for a, b in doors:
        if a not in reachability:
            reachability[a] = set()
        if b not in reachability:
            reachability[b] = set()
        reachability[a].add(b)
        reachability[b].add(a)
    
    return reachability

In [7]:
def paths_to_rooms(reachability):
    consider = deque()
    consider.append((Point(0, 0), set()))
    paths = dict()
    
    while consider:
        location, visited = consider.popleft()
        paths[location] = len(visited)
        for next_step in reachability.get(location, set()):
            if next_step not in visited:
                consider.append((next_step, visited|{next_step}))
    
    return paths

In [8]:
doors = find_all_doors(data)
reach = reachability_dict(doors)
paths = paths_to_rooms(reach)
p1 = max(paths.values())
print(f'Part 1: {p1}')
p2 = sum(1 for dist in paths.values() if dist >= 1000)
print(f'Part 2: {p2}')

Part 1: 3739
Part 2: 8409
